# 라이브러리 불러오기

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1차 모델

## 모델 만들기

### 데이터 불러오기

In [9]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
activity_data = survey[['content_id', 'activity']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
activity_data['content_id'] = activity_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
activity_data = activity_data[activity_data['content_id'] != 0].reset_index().drop(columns=['index'])
activity_data['content_id'] = activity_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(activity_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
activity_content = pd.merge(activity_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
activity_content['activity'] = activity_content['activity'].astype(int)
activity_content

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_17028/1686113110.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_data['content_id'] = activity_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


,content_id,activity,content
0,361617,0,오류장치 ㅠㅠ\n.\n.\n옵으로 다시 참여\n그 땐 문제도 못 풀었지만 100방 ...
1,424772,0,390. 조도 무슨일이야...
2,430139,0,4인. 309번째 심연으로 떠납니다
3,431064,0,4인. 극악의 조도. 그게 다.
4,432520,0,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다.. 조도도 낮은데 관찰..."
...,...,...,...
5018,425802,0,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마^_^
5019,428828,0,첫방 디버프로 초반에 절어버림..ㅋㅋ 볼것도 못보고 힌트썼는데 잘꾸며놓았다 ㅎㅎ 살...
5020,428930,0,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐
5021,429009,0,"265, 2인, 장치...ㅋㅋㅋㅋㅋㅋ 그걸 왜 몰랐냐... 장치에 속지 말자"


In [10]:
# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

activity_content['content'] = activity_content['content'].apply(cleaned_content)

final_df = activity_content[['content', 'activity']]
final_df

,content,activity
0,오류장치 . . . 옵으로 다시 참여. 그 땐 문제도 못 풀었지만 100방 이상 쌓...,0
1,390. 조도 무슨일이야.,0
2,4인. 309번째 심연으로 떠납니다,0
3,4인. 극악의 조도. 그게 다.,0
4,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다. 조도도 낮은데 관찰력...",0
...,...,...
5018,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0
5019,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0
5020,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0
5021,"265, 2인, 장치. 그걸 왜 몰랐냐. 장치에 속지 말자",0


In [11]:
# 라벨별 개수 확인
final_df['activity'].value_counts()

activity
 0    4801
 1     198
-1      24
Name: count, dtype: int64

In [12]:
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['activity'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['activity'].value_counts())

train_data activity
 0    3850
 1     150
-1      18
Name: count, dtype: int64

 test_data activity
 0    970
 1     32
-1      3
Name: count, dtype: int64


In [13]:
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005


중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [ ]:
display(train_data.head()), display(test_data.head())

### 토큰 추가하기 및 사전 학습된 모델과 토큰 불러오기

In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

In [ ]:
# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["activity"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['activity'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './activity_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train()

### 모델 저장하기

In [ ]:
trainer.save_model("./activity_model/try_1/activity_model1")
tokenizer.save_pretrained("./activity_model/try_1/activity_tokenizer1")

### 모델 불러오기

In [14]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./activity_model/try_1/activity_model1"  
tokenizer_path = "./activity_model/try_1/activity_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 모델 예측하기

In [15]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:14<00:00, 71.47it/s]


,content,activity,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,1,1
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [16]:
print(train_data['activity'].value_counts())
print(test_data['activity'].value_counts())

activity
 0    3836
 1     149
-1      18
Name: count, dtype: int64
activity
 0    967
 1     31
-1      3
Name: count, dtype: int64


In [17]:
print(classification_report(test_data['activity'], test_data['pred']))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.99      1.00      1.00       967
           1       0.92      0.77      0.84        31

    accuracy                           0.99      1001
   macro avg       0.64      0.59      0.61      1001
weighted avg       0.99      0.99      0.99      1001



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [18]:
test_data2 = test_data[test_data['activity'] != 0]
test_data2

,content,activity,pred
4,1.5/3인. 활동성 있음.추락조심,1,1
13,"#26, 직원 친절, 공포도 없이 인테리어 잘 구현한 소소한 감성테마, 다양한 종류...",1,1
28,바지추천. 음 무난한 마음이 몽글몽글 해지는 테마다. 후반부에 장치 이슈로 시간이 ...,1,0
46,나름 활동성도 있었던 것 같고 재밌는 장치가 있었음,1,1
138,평소 머팍스타일과 다름. . 장치 다양하고 인테리어도 맘에들었음. 중하라고 되어있으...,1,0
173,많이 돌아다니게 되는 테마 구석구석 잘 살펴보세용.,1,0
200,색다르고 특이한 요소가 있고 할것도 많아서 너무 재밌었어요 볼륨도 제법 큰편이고 활...,1,1
211,#17 2인 플레이 스토리가 좀 유치하긴 했는데 . 나름 잘 꾸며진 테마였습니다. ...,1,1
218,#369. 2인플레이 . 귀엽다. 인테리어도 귀엽다. 이런류의 묘한 귀여운 현실감 ...,1,1
288,3인 5연방 첫번째 의문의 활동성 풀길,1,1


In [19]:
print(classification_report(test_data2['activity'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.00      0.00      0.00         0
           1       0.92      0.77      0.84        31

    accuracy                           0.71        34
   macro avg       0.31      0.26      0.28        34
weighted avg       0.84      0.71      0.77        34



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

In [22]:
test = ['활동성 최악이네', '활동성 개쩔어', '활동성이 안 좋지는 않았어요.', 
'스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30)

문장: 활동성 최악이네 → 평가: 2
------------------------------
문장: 활동성 개쩔어 → 평가: 2
------------------------------
문장: 활동성이 안 좋지는 않았어요. → 평가: 2
------------------------------
문장: 스토리 구려 → 평가: 1
------------------------------
문장: 활동성은 없는데, 인테리어는 좋네요. → 평가: 2
------------------------------


## 1차 결론

- 긍정과 부정 평가가 거의 없어서 학습이 제대로 되지 않는 부분도 있는 것 같다.
- 특히 긍정 평가에 비해서 부정 평가가 현저히 떨어져서 데이터를 증강하는 방법 외에는 성능을 높히는 방안이 없어 보인다.

# 2차 모델

## 추가 사항
- 긍정/부정 데이터 추가
- train/test data 에 직접 추가 방식으로 진행

In [7]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
activity_data = survey[['content_id', 'activity']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
activity_data['content_id'] = activity_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
activity_data = activity_data[activity_data['content_id'] != 0].reset_index().drop(columns=['index'])
activity_data['content_id'] = activity_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(activity_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
activity_content = pd.merge(activity_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
activity_content['activity'] = activity_content['activity'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

activity_content['content'] = activity_content['content'].apply(cleaned_content)

final_df = activity_content[['content', 'activity']]

train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['activity'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['activity'].value_counts())

# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_52456/958464507.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_data['content_id'] = activity_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


train_data activity
 0    3850
 1     150
-1      18
Name: count, dtype: int64

 test_data activity
 0    970
 1     32
-1      3
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005


중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

### 데이터 추가하기

In [ ]:
# 부정 표현 학습을 위해 부정어 데이터 증강
new_data = [('활동성 최악', -1), ('활동성 최악은 아님', 1), ('활동성은 최악이다.', -1), ('활동성은 정말 최악 중에 최악이야.', -1),
('활동성 별로임', -1), ('활동성이 별로는 아니야.', 1), ('활동성 개별로', -1),
('활동성 안 좋음', -1), ('활동성이 안 좋지 않음', 1), ('활동성 개좋음', 1), ('활동성 개쩐다', 1), ('활동성 쩐다', 1),
('활동성 불호', -1), ('활동성 극호', 1), ('활동성 개빡침', -1),
('활동성 불유쾌함', -1), ('활동성 안 유쾌함', -1), ('활동성 극혐', -1), ('활동성 유쾌하지 않음', -1),
('활동성 천상임', 1), ('활동성 생각보다 괜찮음', 1), ('활동성 단순함', -1), 
('활동성 돈 아까움', -1), ('활동성 그닥....', -1), ('활동성 억지스러움', -1),
('치마 비추', 1), ('치마 입으면 불편할 거예요.', 1), ('활동하기 편한 옷 입으세요', 1), ('누워만 있어도 되겠는데', -1),
('움직이지 않아도 된 다는 거죠', -1), ('여긴 누워서 이동하는 곳인듯', -1),
]
new_df = pd.DataFrame(new_data, columns=['content', 'activity'])
train_data = pd.concat([train_data, new_df])
train_data

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["activity"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['activity'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './activity_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train()

In [ ]:
trainer.save_model("./activity_model/try_2/activity_model2")
tokenizer.save_pretrained("./activity_model/try_2/activity_tokenizer2")

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./activity_model/try_2/activity_model2"
tokenizer_path = "./activity_model/try_2/activity_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

In [8]:
new_test_df = [
    ('활동성 개쩌는 테마다', 1),
    ('하루종일 뛰어다닌 듯!', 1),
    ('치마 입고 와서 하루종일 후회함', 1),
    ('누워서 진행하는 방탈출 테마!', -1),
    ('아무것도 한 게 없는 것 같다', -1),
    ('가만히 있다가 가마니 돼서 집에 감', -1)
]
new_test_df = pd.DataFrame(new_test_df, columns=['content', 'activity'])
test_data = pd.concat([test_data, new_test_df])
test_data

,content,activity
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0
1,난.바보야.,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0
4,1.5/3인. 활동성 있음.추락조심,1
...,...,...
1,하루종일 뛰어다닌 듯!,1
2,치마 입고 와서 하루종일 후회함,1
3,누워서 진행하는 방탈출 테마!,-1
4,아무것도 한 게 없는 것 같다,-1


In [9]:
pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data

100%|██████████| 1007/1007 [00:11<00:00, 85.61it/s]


,content,activity,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,1,1
...,...,...,...
1,하루종일 뛰어다닌 듯!,1,1
2,치마 입고 와서 하루종일 후회함,1,0
3,누워서 진행하는 방탈출 테마!,-1,0
4,아무것도 한 게 없는 것 같다,-1,0


In [11]:
print(train_data['activity'].value_counts())
print(test_data['activity'].value_counts())
print('\n')
print(classification_report(test_data['activity'], test_data['pred']))
print('\n')
test_data2 = test_data[test_data['activity'] != 0]
print(classification_report(test_data2['activity'], test_data2['pred']))

activity
 0    3836
 1     149
-1      18
Name: count, dtype: int64
activity
 0    967
 1     34
-1      6
Name: count, dtype: int64


              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         6
           0       0.99      1.00      1.00       967
           1       0.94      0.85      0.89        34

    accuracy                           0.99      1007
   macro avg       0.64      0.62      0.63      1007
weighted avg       0.98      0.99      0.99      1007



              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         6
           0       0.00      0.00      0.00         0
           1       0.94      0.85      0.89        34

    accuracy                           0.73        40
   macro avg       0.31      0.28      0.30        40
weighted avg       0.80      0.72      0.76        40



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [15]:
test = ['활동성 최악이네', '활동성 개쩔어', '활동성이 안 좋지는 않았어요.', 
'스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30, '\n')

문장: 활동성 최악이네 → 평가: 2
------------------------------ 

문장: 활동성 개쩔어 → 평가: 2
------------------------------ 

문장: 활동성이 안 좋지는 않았어요. → 평가: 2
------------------------------ 

문장: 스토리 구려 → 평가: 1
------------------------------ 

문장: 활동성은 없는데, 인테리어는 좋네요. → 평가: 2
------------------------------ 



## 2차 결론
- 부정 표현... 데이터 증강 절실...

# 3차 모델

In [6]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
activity_data = survey[['content_id', 'activity']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
activity_data['content_id'] = activity_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
activity_data = activity_data[activity_data['content_id'] != 0].reset_index().drop(columns=['index'])
activity_data['content_id'] = activity_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(activity_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
activity_content = pd.merge(activity_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
activity_content['activity'] = activity_content['activity'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

activity_content['content'] = activity_content['content'].apply(cleaned_content)

final_df = activity_content[['content', 'activity']]
final_df


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_44271/1686488975.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_data['content_id'] = activity_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


,content,activity
0,오류장치 . . . 옵으로 다시 참여. 그 땐 문제도 못 풀었지만 100방 이상 쌓...,0
1,390. 조도 무슨일이야.,0
2,4인. 309번째 심연으로 떠납니다,0
3,4인. 극악의 조도. 그게 다.,0
4,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다. 조도도 낮은데 관찰력...",0
...,...,...
5018,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0
5019,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0
5020,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0
5021,"265, 2인, 장치. 그걸 왜 몰랐냐. 장치에 속지 말자",0


In [7]:
new_data = [('활동성 최악', -1), ('활동성 최악은 아님', 1), ('활동성은 최악이다.', -1), ('활동성은 정말 최악 중에 최악이야.', -1),
('활동성 별로임', -1), ('활동성이 별로는 아니야.', 1), ('활동성 개별로', -1),
('활동성 안 좋음', -1), ('활동성이 안 좋지 않음', 1), ('활동성 개좋음', 1), ('활동성 개쩐다', 1), ('활동성 쩐다', 1),
('활동성 불호', -1), ('활동성 극호', 1), ('활동성 개빡침', -1),
('활동성 불유쾌함', -1), ('활동성 안 유쾌함', -1), ('활동성 극혐', -1), ('활동성 유쾌하지 않음', -1),
('활동성 천상임', 1), ('활동성 생각보다 괜찮음', 1), ('활동성 단순함', -1), 
('활동성 돈 아까움', -1), ('활동성 그닥....', -1), ('활동성 억지스러움', -1),
('치마 비추', 1), ('치마 입으면 불편할 거예요.', 1), ('활동하기 편한 옷 입으세요', 1), ('누워만 있어도 되겠는데', -1),
('움직이지 않아도 된 다는 거죠', -1), ('여긴 누워서 이동하는 곳인듯', -1), ('활동성 개쩌는 테마다', 1), ('하루종일 뛰어다닌 듯!', 1),
('치마 입고 와서 하루종일 후회함', 1), ('누워서 진행하는 방탈출 테마!', -1),
('아무것도 한 게 없는 것 같다', -1), ('가만히 있다가 가마니 돼서 집에 감', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'activity'])
final_df = pd.concat([final_df, new_df])
final_df

,content,activity
0,오류장치 . . . 옵으로 다시 참여. 그 땐 문제도 못 풀었지만 100방 이상 쌓...,0
1,390. 조도 무슨일이야.,0
2,4인. 309번째 심연으로 떠납니다,0
3,4인. 극악의 조도. 그게 다.,0
4,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다. 조도도 낮은데 관찰력...",0
...,...,...
32,하루종일 뛰어다닌 듯!,1
33,치마 입고 와서 하루종일 후회함,1
34,누워서 진행하는 방탈출 테마!,-1
35,아무것도 한 게 없는 것 같다,-1


In [8]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(kiwi_clean)

In [9]:
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['activity'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['activity'].value_counts())

# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

train_data activity
 0    3852
 1     159
-1      37
Name: count, dtype: int64

 test_data activity
 0    943
 1     29
-1      3
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4048
중복 제거 전 테스트 데이터셋: 975


중복 제거 후 학습 데이터셋: 3945
중복 제거 후 테스트 데이터셋: 959


In [28]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [29]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["activity"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['activity'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './activity_model/check_point/try_3',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [30]:
# train 진행
trainer.train()

  0%|          | 0/7410 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_17028/739193953.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7410 [01:28<20:17,  5.68it/s]

{'loss': 0.1947, 'learning_rate': 4.662618083670716e-05, 'epoch': 2.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_17028/739193953.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 13%|█▎        | 1000/7410 [02:56<18:30,  5.77it/s]

{'loss': 0.0661, 'learning_rate': 4.3252361673414306e-05, 'epoch': 4.05}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_17028/739193953.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|██        | 1500/7410 [04:24<17:27,  5.64it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_17028/739193953.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 94%|█████████▍| 7000/7410 [20:30<01:12,  5.69it/s]

{'loss': 0.0196, 'learning_rate': 2.766531713900135e-06, 'epoch': 28.34}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_17028/739193953.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7410/7410 [21:42<00:00,  5.69it/s]

{'train_runtime': 1302.5375, 'train_samples_per_second': 90.861, 'train_steps_per_second': 5.689, 'train_loss': 0.047548150758833375, 'epoch': 30.0}


TrainOutput(global_step=7410, training_loss=0.047548150758833375, metrics={'train_runtime': 1302.5375, 'train_samples_per_second': 90.861, 'train_steps_per_second': 5.689, 'train_loss': 0.047548150758833375, 'epoch': 30.0})

In [31]:
trainer.save_model("./activity_model/try_3/activity_model3")
tokenizer.save_pretrained("./activity_model/try_3/activity_tokenizer3")

('./activity_model/try_3/activity_tokenizer3/tokenizer_config.json',
 './activity_model/try_3/activity_tokenizer3/special_tokens_map.json',
 './activity_model/try_3/activity_tokenizer3/vocab.txt',
 './activity_model/try_3/activity_tokenizer3/added_tokens.json')

In [2]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./activity_model/try_3/activity_model3"
tokenizer_path = "./activity_model/try_3/activity_tokenizer3"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

In [14]:
pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data

100%|██████████| 959/959 [00:11<00:00, 85.77it/s]


,content,activity,pred
0,인 장치 오류 아쉽다 재미있다,0,0
1,이다 장치 오류 많다 실망,0,0
2,장치 오류 그래도 참신,0,0
3,방 탈출 너무 중독 방 태기 좀 오다 사람 드리다,0,0
4,이 방 탈출 트랙 션 보다 맞다 문제 요소 전무 보다 되다 정도,0,0
...,...,...,...
969,나 너무 어렵다 방탈짬바 있다 추천,0,0
970,인 각자 하다 뿌듯하다 테마,0,0
971,첫 디버프로 초반 절다 보다 못 보다 힌트 쓰다 잘 꾸미다 살짝 장치 오류 아쉽다,0,0
972,인 볼륨 압도 히터 방 빵빵하다 다른 테마 궁금,0,0


In [19]:
print('0포함 확률', classification_report(test_data['activity'], test_data['pred']))
print('\n')
test_data2 = test_data[test_data['activity'] != 0]
print('0제외 확률', classification_report(test_data2['activity'], test_data2['pred']))

0포함 확률               precision    recall  f1-score   support

          -1       0.50      0.33      0.40         3
           0       1.00      1.00      1.00       928
           1       0.83      0.89      0.86        28

    accuracy                           0.99       959
   macro avg       0.78      0.74      0.75       959
weighted avg       0.99      0.99      0.99       959



0제외 확률               precision    recall  f1-score   support

          -1       1.00      0.33      0.50         3
           0       0.00      0.00      0.00         0
           1       0.93      0.89      0.91        28

    accuracy                           0.84        31
   macro avg       0.64      0.41      0.47        31
weighted avg       0.93      0.84      0.87        31



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [16]:
test = ['활동성 최악이네', '활동성 개쩔어', '활동성이 안 좋지는 않았어요.', 
'스토리 구려', '활동성은 없는데, 인테리어는 좋네요.']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30)

문장: 활동성 최악이네 → 평가: 0
------------------------------
문장: 활동성 개쩔어 → 평가: 2
------------------------------
문장: 활동성이 안 좋지는 않았어요. → 평가: 0
------------------------------
문장: 스토리 구려 → 평가: 1
------------------------------
문장: 활동성은 없는데, 인테리어는 좋네요. → 평가: 2
------------------------------
